<a href="https://colab.research.google.com/github/Olivernyu/Project_CSUY_4613/blob/main/FinetuneLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
from transformers import set_seed

# Set random seeds
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
set_seed(seed)

In [ ]:
import pandas as pd
from datasets import Dataset

def load_jigsaw_dataset(train_path, test_path, test_labels_path):
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    test_labels_df = pd.read_csv(test_labels_path)

    # Merge the test dataset with its labels
    test_df = test_df.merge(test_labels_df, on='id')

    # Filter out rows with -1 labels (unlabeled samples)
    test_df = test_df[test_df['toxic'] != -1]

    # Load as Hugging Face Dataset
    train_dataset = Dataset.from_pandas(train_df[['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']])
    test_dataset = Dataset.from_pandas(test_df[['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']])

    return train_dataset, test_dataset

# Load the Jigsaw dataset
train_dataset, test_dataset = load_jigsaw_dataset('train.csv', 'test.csv', 'test_labels.csv')


# Load the Jigsaw dataset


In [ ]:
# skip
# from transformers import AutoTokenizer
# import torch

# # Load the pre-trained tokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # Tokenize the dataset
# # def tokenize(batch):
# #     return tokenizer(batch['comment_text'], padding="max_length", truncation=True, max_length=512)
# # def tokenize(batch):
# #     labels = [batch[col] for col in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
# #     labels = torch.tensor(labels).float()
# #     return tokenizer(batch['comment_text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt").update({"labels": labels})
# def tokenize(batch):
#     labels = [[batch[col][i] for col in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] for i in range(len(batch['comment_text']))]
#     labels = torch.tensor(labels).float()
#     tokenized_batch = tokenizer(batch['comment_text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#     tokenized_batch["labels"] = labels
#     return tokenized_batch

# # Need to process at small batchsize due to memory limitation in google colab, used batch_size=32 here
# train_dataset = train_dataset.map(tokenize, batched=True, batch_size=32)
# train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
# test_dataset = test_dataset.map(tokenize, batched=True, batch_size=32) 
# test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])




In [ ]:
from torch.nn import BCEWithLogitsLoss
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize(batch):
    labels = [[batch[col][i] for col in ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] for i in range(len(batch['comment_text']))]
    labels = torch.tensor(labels).float()
    tokenized_batch = tokenizer(batch['comment_text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    tokenized_batch["labels"] = labels
    return tokenized_batch

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=32)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

test_dataset = test_dataset.map(tokenize, batched=True, batch_size=32)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])




Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Map:   0%|          | 0/63978 [00:00<?, ? examples/s]

In [ ]:

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

# Training arguments
training_args = TrainingArguments(
    output_dir="toxicity_detection",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_dir="logs",
)

# Custom Trainer class for multi-label classification
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

# Instantiate the custom Trainer
trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
500,0.114000
1000,0.058900
1500,0.055400
2000,0.052700
2500,0.051800
3000,0.046600
3500,0.046600
4000,0.051400
4500,0.047900
5000,0.040200
